<a href="https://colab.research.google.com/github/otwn/Geospatial-Analysis-Examples/blob/master/Interactive_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive Map

This is based on Kaggle's Geospatial Analysis course. They use folium but here I use geemap and google earth engine

* [geemap](https://github.com/giswqs/geemap)
* google earth engine

In [ ]:
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

try:
    import geopandas
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geopandas'])


geemap package not installed. Installing ...
geemap package not installed. Installing ...


**Important note:** A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only (source). Note that Google Colab currently does not support ipyleaflet (source). Therefore, if you are using geemap with Google Colab, you should use import geemap.eefolium. If you are using geemap with binder or a local Jupyter notebook server, you can use import geemap, which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [ ]:
import pandas as pd
import geopandas as gpd

try: 
  import google.colab
  import geemap.eefolium as emap
except:
  import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()    

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=FIXoL9z4xT7Vlp48FgWKeM4s1zaWJdyXAadzQZzFg-4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/3gEFiV1Qj2Na2Pk9udwf9u4HpQBSut9t-HXq5vXUuLfBM8EEWurzcUM

Successfully saved authorization token.


In [ ]:
m_1 = emap.Map(center=[42.32, -71.0589], zoom=10)
m_1.add_basemap('ESRI Ocean')
m_1

Basemap can only be one of the following: ROADMAP, SATELLITE, TERRAIN, HYBRID, ESRI, Esri Ocean, Esri Satellite, Esri Standard, Esri Terrain, Esri Transportation, Esri Topo World, Esri National Geographic, Esri Shaded Relief, Esri Physical Map, Bing VirtualEarth, 3DEP Elevation, NAIP Imagery


# Add Marker
* crimes in Boston
* in 2018
* daytime (9am-6pm) crimes

In [ ]:
crimes_boston = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Geospatial/crimes-in-boston/crime.csv", encoding="latin-1")

# data cleaning
crimes_boston.dropna(subset=["Lat","Long", "DISTRICT"], inplace=True)
crimes_boston = crimes_boston[crimes_boston.OFFENSE_CODE_GROUP.isin(['Larceny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary', 
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment', 
    'Manslaughter'])]

# focus on 2018
crimes_boston_2018 = crimes_boston[crimes_boston.YEAR>=2018]

crimes_boston_2018.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
6,I182070933,724,Auto Theft,AUTO THEFT,B2,330,NaN,2018-09-03 21:25:00,2018,9,Monday,21,Part One,NORMANDY ST,42.306072,-71.082733,"(42.30607218, -71.08273260)"
8,I182070931,301,Robbery,ROBBERY - STREET,C6,177,NaN,2018-09-03 20:48:00,2018,9,Monday,20,Part One,MASSACHUSETTS AVE,42.331521,-71.070853,"(42.33152148, -71.07085307)"
19,I182070915,614,Larceny From Motor Vehicle,LARCENY THEFT FROM MV - NON-ACCESSORY,B2,181,NaN,2018-09-02 18:00:00,2018,9,Sunday,18,Part One,SHIRLEY ST,42.325695,-71.068168,"(42.32569490, -71.06816778)"
24,I182070908,522,Residential Burglary,BURGLARY - RESIDENTIAL - NO FORCE,B2,911,NaN,2018-09-03 18:38:00,2018,9,Monday,18,Part One,ANNUNCIATION RD,42.335062,-71.093168,"(42.33506218, -71.09316781)"


In [ ]:
# Focus on crimes between 9am - 6pm
daytime_robberies = crimes_boston_2018[((crimes_boston_2018.OFFENSE_CODE_GROUP=="Robbery") & (crimes_boston_2018.HOUR.isin(range(9,18))))]
daytime_robberies.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
299,I182070598,311,Robbery,ROBBERY - COMMERCIAL,A15,60,NaN,2018-09-02 17:15:00,2018,9,Sunday,17,Part One,RUTHERFORD AVE,42.371673,-71.063264,"(42.37167264, -71.06326413)"
527,I182070342,381,Robbery,ROBBERY - CAR JACKING,E18,490,NaN,2018-09-01 17:05:00,2018,9,Saturday,17,Part One,CUMMINS HWY,42.276453,-71.112980,"(42.27645319, -71.11297971)"
562,I182070309,361,Robbery,ROBBERY - OTHER,D4,151,NaN,2018-09-01 15:00:00,2018,9,Saturday,15,Part One,BERKELEY ST,42.346055,-71.070507,"(42.34605510, -71.07050669)"
1127,I182069693,301,Robbery,ROBBERY - STREET,A1,74,NaN,2018-08-30 15:24:00,2018,8,Thursday,15,Part One,BULFINCH PL,42.362191,-71.062238,"(42.36219081, -71.06223802)"
1135,I182069687,301,Robbery,ROBBERY - STREET,B2,278,NaN,2018-08-28 12:00:00,2018,8,Tuesday,12,Part One,HARRISON AVE,42.331538,-71.080157,"(42.33153805, -71.08015661)"


In [ ]:
m_2 = emap.Map(location=[42.32, -71.0589], zoom_start=13)

for idx, row in daytime_robberies.iterrows():
  emap.folium.Marker([row["Lat"], row["Long"]]).add_to(m_2)

m_2

# Show crimes points using MarkerCluster

In [ ]:
import math

m_3 = emap.Map(center=[42.32, -71.0589], zoom=13)

# add points as cluster to the map
mc = emap.folium.plugins.MarkerCluster()
for idx, row in daytime_robberies.iterrows():
  if not math.isnan(row["Long"]) and not math.isnan(row["Lat"]):
    mc.add_child(emap.folium.Marker([row["Lat"], row["Long"]]))

m_3.add_child(mc)
m_3

# Bubble maps

In [ ]:
m_4 = emap.Map(center=[42.32, -71.0589], zoom=13, add_google_map=False, tiles='cartodbpositron')

def color_producer(val):
  if val <= 12:
    return "forestgreen"
  else:
    return "darkred"

for i in range(0, len(daytime_robberies)):
  emap.folium.Circle(
      location=[daytime_robberies.iloc[i]["Lat"],daytime_robberies.iloc[i]["Long"]],
      radius=20,
      color=color_producer(daytime_robberies.iloc[i]["HOUR"])
  ).add_to(m_4)

m_4

# Heatmap


In [ ]:
m_5 = emap.Map(center=[42.32,-71.0589], zoom=12, add_google_map=False, tiles='cartodbpositron')

emap.folium.plugins.HeatMap(data=daytime_robberies[["Lat", "Long"]], radius=10).add_to(m_5)

m_5

In [ ]:
# In Boston in 2018
m_6 = emap.Map(center=[42.32,-71.0589], zoom=12, add_google_map=False, tiles='cartodbpositron')

emap.folium.plugins.HeatMap(data=crimes_boston_2018[["Lat", "Long"]], radius=10).add_to(m_6)

m_6

# Chrolopleth maps with GeoPandas

In [ ]:
districts_full = gpd.read_file("/content/drive/My Drive/Colab Notebooks/Geospatial/Police_Districts/Police_Districts.shp")
districts= districts_full[["DISTRICT","geometry"]].set_index("DISTRICT")
districts.head()

,geometry
DISTRICT,
A15,"MULTIPOLYGON (((-71.07416 42.39051, -71.07415 ..."
A7,"MULTIPOLYGON (((-70.99644 42.39557, -70.99644 ..."
A1,"POLYGON ((-71.05200 42.36884, -71.05169 42.368..."
C6,"POLYGON ((-71.04406 42.35403, -71.04412 42.353..."
D4,"POLYGON ((-71.07416 42.35724, -71.07359 42.357..."


In [ ]:
plot_dict = crimes_boston_2018.DISTRICT.value_counts()
plot_dict.head()

D4     2885
B2     2231
A1     2130
C11    1899
B3     1421
Name: DISTRICT, dtype: int64

In [ ]:
m_7 = emap.Map(center=[42.32, -71.0589], zoom=12, add_google_map=False, tiles="cartodbpositron")

emap.folium.Choropleth(geo_data=districts.__geo_interface__,
                       data=plot_dict,
                       key_on="feature.id",
                       fill_color="YlGnBu",
                       legend_name="Major criminal incidents (Jan-Aug 2018)").add_to(m_7)

m_7

# Overlay Sattelite Image

In [ ]:
image = ee.Image("USGS/SRTMGL1_003")
vis_params={
    'min':0,
    'max':4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
    }
m_1.addLayer(image,  vis_params, 'SRTM DEM', True, 0.5)
m_1